In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [16]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_1월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(705297, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
705297


In [17]:
df

,split_str,org_idx,morphs,pos_tag
0,방문한지 4달이 조금 안된시점에 포스팅을 하게되었다,0,"[방문, 한지, 4, 달이, 조금, 안된, 시점, 에, 포스팅, 을, 하게, 되었다]","[('방문', 'Noun'), ('한지', 'Noun'), ('4', 'Number..."
1,사실은 밀려있는 그릿비에게 밀려서 잊고 있었던 코랄라니 카페 공사하는모습은 진작에 ...,0,"[사실, 은, 밀려있는, 그릿비, 에게, 밀려서, 잊고, 있었던, 코랄, 라니, 카...","[('사실', 'Noun'), ('은', 'Josa'), ('밀리다', 'Verb'..."
2,대형카페의 규모는 도대체 얼마나 더 커질예정인지 참 가늠이 안되는구나 라는 생각을 ...,0,"[대형, 카페, 의, 규모, 는, 도대체, 얼마나, 더, 커질, 예정, 인지, 참,...","[('대형', 'Noun'), ('카페', 'Noun'), ('의', 'Josa')..."
3,근래 생긴 기장에 속해있는 카페중에 주차장이 가장 잘되어있다고 생각 수용인원에대한 ...,0,"[근래, 생긴, 기장, 에, 속, 해있는, 카페, 중, 에, 주차장, 이, 가장, ...","[('근래', 'Noun'), ('생기다', 'Verb'), ('기장', 'Noun..."
4,오픈 일자에 방문했는데도 미리 걸려있던 현수막 덕분인지 주차장을 들어가는데 한참을 ...,0,"[오픈, 일자, 에, 방문, 했는데도, 미리, 걸려있던, 현수막, 덕분, 인지, 주...","[('오픈', 'Noun'), ('일자', 'Noun'), ('에', 'Josa')..."
...,...,...,...,...
705292,일 년 중 한 달을 아주 진하게 찾아오는 우울에 적셔지고 있노라면 blog nave...,13119,"[일, 년, 중, 한, 달, 을, 아주, 진하게, 찾아오는, 우울, 에, 적셔지고,...","[('일', 'Noun'), ('년', 'Noun'), ('중', 'Noun'), ..."
705293,1984 동물농장 을 읽을 때도 느꼈다 옛날 사람이 자신의 시대를 신랄하게 비판한 ...,13119,"[1984, 동물농장, 을, 읽을, 때, 도, 느꼈다, 옛날, 사람, 이, 자신, ...","[('1984', 'Number'), ('동물농장', 'Noun'), ('을', '..."
705294,나는 나 자신이 되면 된다 인간이라면 으레 경험하는 감정을 유순한 마음으로 받아들이...,13119,"[나, 는, 나, 자신, 이, 되면, 된다, 인간, 이, 라면, 으레, 경험, 하는...","[('나', 'Noun'), ('는', 'Josa'), ('나', 'Noun'), ..."
705295,끼야홍 독후감을 좀 더 고심해서 적어야겠어,13119,"[끼, 야홍, 독후감, 을, 좀, 더, 고심, 해서, 적어야겠어]","[('끼', 'Noun'), ('야홍', 'Noun'), ('독후감', 'Noun'..."


In [26]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [24]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/1월/나무문화_1월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(140934, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '사람', '곳', '때', '등', '우리', '말', '나', '내', '생각', '더', '의', '문화', '위', '길', '일', '시간', '때문', '제', '나무', '안', '중', '전', '사진', '책', '하나', '집', '위해', '글', '한국', '다른', '속', '를', '마음', '아이', '작품', '마을', '대한', '시작', '여행', '날', '가지', '저', '개', '자신', '공간', '또']


In [28]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 705297/705297 [00:05<00:00, 120706.85it/s]


In [30]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

12
12


In [31]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 705297/705297 [08:34<00:00, 1372.13it/s] 


In [35]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [41]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/1월/단어동시출현_딕셔너리_1월.csv", encoding='UTF-8')

In [43]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 141079090/141079090 [14:24<00:00, 163125.95it/s]


(141079090, 3)


,term1,term2,freq
0,달이,조금,13
1,되었다,조금,123
2,4,조금,149
3,에,조금,3453
4,조금,한지,10


In [45]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/1월/단어동시출현_df_1월.csv", index=False, encoding='UTF-8')

In [46]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 141079090/141079090 [26:36<00:00, 88362.84it/s]


,term1,term2,freq
0,곳,이,30011
1,것,이,86530
2,생각,이,19577
3,의,이,158520
4,더,이,17370
...,...,...,...
1220,여행,위해,382
1221,마을,위,593
1222,마을,생각,394
1223,마을,저,413


In [47]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/1월/단어동시출현_df_상위50_1월.csv", index=False, encoding='UTF-8')

In [48]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1507.84it/s]


In [50]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,사람,곳,때,등,우리,말,나,내,생각,더,의,문화,위,길,일,시간,때문,제,나무,안,중,전,사진,책,하나,집,위해,글,한국,다른,속,를,마음,아이,작품,마을,대한,시작,여행,날,가지,저,개,자신,공간,또
것,0.0,23259.0,24510.0,86530.0,14237.0,7257.0,11757.0,6728.0,10962.0,11922.0,12340.0,10084.0,10090.0,8306.0,66352.0,4157.0,6351.0,3295.0,7982.0,4931.0,7797.0,4296.0,3746.0,6424.0,5711.0,5181.0,2584.0,3663.0,4933.0,3233.0,4734.0,2189.0,2733.0,5626.0,5287.0,49756.0,4885.0,2929.0,2852.0,1660.0,5014.0,3640.0,2294.0,3130.0,4679.0,4087.0,2426.0,5733.0,2288.0,4039.0
수,23259.0,0.0,13417.0,49892.0,8406.0,7916.0,6649.0,7607.0,6570.0,6016.0,7926.0,6161.0,5753.0,5846.0,44860.0,4469.0,4272.0,3122.0,5092.0,4524.0,4996.0,3569.0,3399.0,4051.0,4015.0,4068.0,2679.0,2679.0,2943.0,2420.0,2930.0,1594.0,2143.0,3640.0,3681.0,37806.0,3190.0,2883.0,2721.0,1836.0,3163.0,2161.0,2237.0,2125.0,3189.0,2441.0,2483.0,3232.0,3950.0,2475.0
그,24510.0,13417.0,0.0,50340.0,10477.0,6940.0,8630.0,4673.0,6558.0,7670.0,8063.0,6365.0,5258.0,4501.0,48697.0,2790.0,4574.0,2540.0,5101.0,2924.0,4646.0,3036.0,2419.0,4621.0,5240.0,3913.0,1595.0,2428.0,3173.0,2683.0,3186.0,1509.0,1862.0,3305.0,4346.0,35992.0,3122.0,1722.0,2446.0,1293.0,3325.0,2603.0,1271.0,2593.0,2959.0,2321.0,1771.0,3858.0,1533.0,2889.0
이,86530.0,49892.0,50340.0,0.0,33287.0,30011.0,27685.0,22026.0,20788.0,22870.0,26089.0,20926.0,19577.0,17370.0,158520.0,12010.0,14377.0,10238.0,16140.0,13458.0,15382.0,12572.0,11299.0,15602.0,15058.0,14897.0,9695.0,9552.0,10403.0,9654.0,9979.0,6706.0,7888.0,11055.0,11489.0,115091.0,10727.0,8566.0,8592.0,7029.0,9651.0,9553.0,6460.0,10670.0,9507.0,10793.0,8261.0,9857.0,8722.0,9240.0
사람,14237.0,8406.0,10477.0,33287.0,0.0,3564.0,4655.0,2242.0,3715.0,5119.0,5422.0,3936.0,4075.0,3053.0,24410.0,1620.0,2344.0,1549.0,3560.0,1815.0,2675.0,1542.0,1525.0,2696.0,2336.0,1970.0,1357.0,1510.0,1756.0,1578.0,2003.0,1045.0,1013.0,3216.0,2168.0,19106.0,2259.0,936.0,872.0,1156.0,1608.0,1364.0,914.0,1392.0,1811.0,1792.0,1046.0,2655.0,984.0,1687.0
곳,7257.0,7916.0,6940.0,30011.0,3564.0,0.0,3239.0,3002.0,2068.0,1764.0,3180.0,2574.0,2176.0,2266.0,18500.0,2427.0,1770.0,2487.0,1675.0,2155.0,1480.0,1913.0,1829.0,2145.0,2481.0,2092.0,2061.0,795.0,1424.0,1616.0,1231.0,689.0,1067.0,1839.0,1432.0,14347.0,1342.0,1906.0,680.0,2215.0,679.0,1157.0,2627.0,1364.0,959.0,1741.0,1275.0,603.0,1846.0,1276.0
때,11757.0,6649.0,8630.0,27685.0,4655.0,3239.0,0.0,2407.0,3408.0,3628.0,4604.0,3805.0,3122.0,2603.0,20137.0,1226.0,1893.0,1343.0,2742.0,1966.0,1993.0,1908.0,1341.0,2506.0,2139.0,2198.0,1258.0,1203.0,1581.0,1620.0,1545.0,763.0,993.0,1606.0,1628.0,17902.0,1729.0,1202.0,778.0,720.0,1222.0,1645.0,1010.0,1595.0,1432.0,1590.0,968.0,1453.0,659.0,1497.0
등,6728.0,7607.0,4673.0,22026.0,2242.0,3002.0,2407.0,0.0,1467.0,1864.0,3040.0,2015.0,1131.0,1293.0,22879.0,3208.0,2552.0,1214.0,1726.0,1205.0,1251.0,2360.0,1987.0,1396.0,2272.0,2562.0,977.0,993.0,876.0,1020.0,1586.0,670.0,1725.0,1120.0,1487.0,15733.0,605.0,867.0,1437.0,1161.0,1600.0,1184.0,768.0,667.0,1216.0,804.0,1910.0,748.0,1594.0,849.0
우리,10962.0,6570.0,6558.0,20788.0,3715.0,2068.0,3408.0,1467.0,0.0,3168.0,3105.0,2433.0,2710.0,2347.0,19936.0,1876.0,1794.0,1213.0,2304.0,1756.0,1875.0,1056.0,1125.0,2317.0,1571.0,1566.0,901.0,1113.0,1364.0,1714.0,1671.0,691.0,906.0,1429.0,1950.0,14917.0,1605.0,1334.0,627.0,638.0,1381.0,1034.0,726.0,1225.0,1348.0,1060.0,742.0,1421.0,666.0,1183.0
말,11922.0,6016.0,7670.0,22870.0,5119.0,1764.0,3628.0,1864.0,3168.0,0.0,4216.0,3630.0,2638.0,2010.0,18710.0,1054.0,1769.0,922.0,2257.0,1128.0,1718.0,1453.0,898.0,2163.0,1623.0,1611.0,631.0,1172.0,1308.0,1026.0,1162.0,936.0,828.0,1468.0,1600.0,14193.0,1558.0,894.0,527.0,453.0,1149.0,968.0,490.0,979.0,1315.0,1221.0,804.0,1711.0,440.0,1245.0


In [51]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/1월/공출현빈도_df_1월.csv", encoding='UTF-8')